In [1]:

%matplotlib widget
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date
import json
import numpy as np
#print(os.getcwd())

In [2]:
ET_DATA_PATH = 'D:\_MasterArbeit_EEDA\EyeTracking'
SCENE_EVENT_DATA_PATH = 'D:\_MasterArbeit_EEDA\SceneData'
STEERING_DATA_PATH = 'D:\_MasterArbeit_EEDA\Input'
PARTICIPANT_DATA_PATH = 'D:\_MasterArbeit_EEDA\ParticipantCalibrationData'
PROCESSED_DATA_PATH = '..\csv_et'
PLOT_PATH = '..\PLOTS\data_quality'
os.makedirs(os.path.dirname(PROCESSED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

### Filter EyeValidationError under 1.5

In [3]:
df = pd.DataFrame()

files = glob.glob(f'{PARTICIPANT_DATA_PATH}/*.txt')

for fi in files[:4]: # reading all files
    with open(fi, 'r') as j:
        tmp = json.loads(j.read())
        tempdf0 = pd.DataFrame.from_dict(tmp,orient='index').T        
        tempdf0[["EyeValidationError.x","EyeValidationError.y","EyeValidationError.z"]] = pd.DataFrame(tempdf0["EyeValidationError"].to_list(), columns=['x', 'y','z'])
        if(tempdf0["EyeValidationError.x"][0]<1.5) and (tempdf0["EyeValidationError.y"][0]<1.5):
            df = pd.concat([df, tempdf0], ignore_index=True)
        
df = df.drop(['IPAddress', 'VRmode',
       'SteeringInputDevice', 'EyeValidationError', 'SeatCalibrationOffset',
       'TrainingDuration', 'ExperimentDuration', 'ApplicationDuration',
       'AverageExperimentFPS', 'SpecialNotes', 'EyeValidationError.z'], axis=1)
display(df)

,ParticipantUuid,ExperimentalCondition,EyeValidationError.x,EyeValidationError.y
0,489749a7e3394cdda22f5f2717a340a8,ManualDriving,0.516909,0.679336


### Check if Incomplete

In [4]:
files = glob.glob(f'{ET_DATA_PATH}/*.txt')
des_index = []
for i in range(0,df['ParticipantUuid'].count()):
    for fi in files[:]: # reading all files
        #print (df['ParticipantUuid'][i])
        if '_Incomplete' in fi:
            if df['ParticipantUuid'][i] in fi:
                if os.path.getsize(fi) < 10 : 
                    des_index.append(i)
                    
                
df = df.iloc[des_index] 
df = df.reset_index(drop=True)
display(df)

,ParticipantUuid,ExperimentalCondition,EyeValidationError.x,EyeValidationError.y
0,489749a7e3394cdda22f5f2717a340a8,ManualDriving,0.516909,0.679336


### Load Eyetracking Data

In [ ]:
files = glob.glob(f'{ET_DATA_PATH}/*.txt')
for i in range(0,df['ParticipantUuid'].count()):
    et_df = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            with open(fi, 'r') as j:
                tmp = json.loads(j.read())
            for nS, sample in enumerate(tmp):
                tmpdf = pd.json_normalize(data=sample, sep='_')
                tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
                tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
                tmpdf = tmpdf.drop(['FPS','pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight',
       'EyeBlinkingWorldLeftTobii', 'EyeBliningWorldRightTobii',
       'EyeBlinkingLocalLeftTobii', 'EyeBlinkingLocalRightTobii', 'hitObjects',
       'HmdRotation_x', 'HmdRotation_y',
       'HmdRotation_z', 'HmdRotation_w','EyePositionLeftWorld_x',
       'EyePositionLeftWorld_y', 'EyePositionLeftWorld_z',
       'EyeDirectionLeftWorld_x', 'EyeDirectionLeftWorld_y',
       'EyeDirectionLeftWorld_z', 'EyePositionLeftLocal_x',
       'EyePositionLeftLocal_y', 'EyePositionLeftLocal_z',
       'EyeDirectionLeftLocal_x', 'EyeDirectionLeftLocal_y',
       'EyeDirectionLeftLocal_z', 'EyePositionRightWorld_x',
       'EyePositionRightWorld_y', 'EyePositionRightWorld_z',
       'EyePositionRightLocal_x', 'EyePositionRightLocal_y',
       'EyePositionRightLocal_z', 'EyeDirectionRightWorld_x',
       'EyeDirectionRightWorld_y', 'EyeDirectionRightWorld_z',
       'EyeDirectionRightLocal_x', 'EyeDirectionRightLocal_y',
       'EyeDirectionRightLocal_z', 'EyePositionWorldCombinedTobii_x',
       'EyePositionWorldCombinedTobii_y', 'EyePositionWorldCombinedTobii_z',
       'EyePositionLocalCombinedTobii_x', 'EyePositionLocalCombinedTobii_y',
       'EyePositionLocalCombinedTobii_z', 'EyeDirectionWorldCombinedTobii_x',
       'EyeDirectionWorldCombinedTobii_y', 'EyeDirectionWorldCombinedTobii_z',
       'EyeDirectionLocalCombinedTobii_x', 'EyeDirectionLocalCombinedTobii_y',
       'EyeDirectionLocalCombinedTobii_z'],axis=1)
                
                et_df = pd.concat([et_df, tmpdf], ignore_index=True)
                   
    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['timestamp'] = pd.to_datetime(et_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        path = os.path.join(PROCESSED_DATA_PATH,df['ParticipantUuid'][i]+'.csv')
        print(100/df['ParticipantUuid'].count()*(i+1),"%")
        et_df.set_index('timestamp', inplace=True)
        et_df['time_from_start'] = (et_df
#                                    .reset_index()
                                   .groupby(['uid','scene',], as_index=False)
#                                    .timestamp
                                   .transform(lambda x: 
                                              (x.index - x.index[0])/np.timedelta64(1,'s'))
                                   .UnixTimeStamp
                                   )
        et_df = et_df.reset_index()
        et_df[['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']]=et_df[['EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y', 'EyeDirectionCombinedLocal_z']]
        et_df['is_blink'] = ((et_df['EyeOpennessLeftSranipal'] < 0.1)&(et_df['EyeOpennessRightSranipal'] < 0.1))
        et_df.loc[(et_df['is_blink'] == True),['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']] = np.nan
        
        tmpdf = (et_df
                 .groupby(['uid', 'scene'])['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']
                 .transform(lambda x: 
                 x.interpolate(method='polynomial', order=3, limit_direction ='both'))
                 .reset_index()
                )
        et_df[['combinedEyeDir_x_intp', 'combinedEyeDir_y_intp', 'combinedEyeDir_z_intp']] = tmpdf[['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']]


        et_df.to_csv(path)

### Steeringwheel Data read and write

In [8]:
files = glob.glob(f'{STEERING_DATA_PATH}/*.txt')
for i in range(0,df['ParticipantUuid'].count()):
    et_df = pd.DataFrame()
    for fi in files[:]: # reading all files
        if df['ParticipantUuid'][i] in fi:
            print("asdw")
            with open(fi, 'r') as j:
                tmp = json.loads(j.read())
            for nS, sample in enumerate(tmp):
                tmpdf = pd.json_normalize(data=sample, sep='_')
                tmpdf['uid'] = fi.split('\\')[-1].split('_')[0]
                tmpdf['scene'] = fi.split('_')[-1].split('.')[0]
                et_df = pd.concat([et_df, tmpdf], ignore_index=True)
                #print(sample)
    
    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['timestamp'] = pd.to_datetime(et_df.TimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        path = os.path.join(PROCESSED_DATA_PATH,df['ParticipantUuid'][i]+'_st_df'+'.csv')
        print(100/df['ParticipantUuid'].count()*(i+1),"%")

#display(et_df)

asdw
asdw
asdw
asdw
asdw
100.0 %


In [ ]:
files = glob.glob(f'{PROCESSED_DATA_PATH}\*.csv')
et_int_df = pd.DataFrame()
for fi in files[:]:
    et_int_df = pd.read_csv(fi)
    ax = et_int_df.plot(kind="scatter", x="time_from_start",y="EyeOpennessLeftSranipal", color="b", label="Left")
    et_int_df.plot(kind="scatter",x="time_from_start",y="EyeOpennessRightSranipal", color="r", label="Right", ax=ax, alpha=0.4)